<a href="https://colab.research.google.com/github/harshalkale/Outskill-GenAIEngineering-Cohort3/blob/main/BaseCamp4/Week2Session2/2_ratings_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Open on Google Colab:

On HuggingFace: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

## Libraries Explained

- **dotenv**: Loads environment variables from a `.env` file into the application's environment, helping manage configuration separately from code.

- **huggingface_hub**:
  - **HfApi**: Provides programmatic access to the Hugging Face model hub for uploading, downloading, and managing models.
  - **hf_hub_download**: Simplifies downloading model files from the Hugging Face hub to your local environment.

- **transformers**: Offers pre-trained models for natural language processing tasks. The `pipeline` function specifically provides an easy-to-use interface for common NLP tasks like text generation, sentiment analysis, and question answering.


In [ ]:
import os, json, datetime
from datetime import datetime
# from dotenv import load_dotenv

from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download



from transformers import pipeline


# Hugging Face Model Reference

[nlptown/bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment)

# Using BERT Base Multilingual Uncased Sentiment Model

This code references a pre-trained multilingual sentiment analysis model from Hugging Face's model hub:

### Model Details
- **Model ID**: `nlptown/bert-base-multilingual-uncased-sentiment`
- **Architecture**: BERT (Bidirectional Encoder Representations from Transformers)
- **Type**: Base variant (110M parameters)
- **Case**: Uncased (text is converted to lowercase during processing)
- **Languages**: Multilingual (supports multiple languages)
- **Task**: Sentiment analysis (predicts sentiment of text)

### Model Capabilities
- Analyzes text sentiment on a 5-star scale (1-5 rating)
- Works across multiple languages (trained on product reviews in 6 languages)
- Can be used for zero-shot sentiment classification in languages beyond English
- Returns star ratings from 1 (very negative) to 5 (very positive)


The model is particularly useful for multilingual applications where sentiment analysis needs to work across different languages without training separate models for each language.

In [ ]:
hf_reference='nlptown/bert-base-multilingual-uncased-sentiment'


# Downloading Specific Model Files from Hugging Face Hub


This code snippet demonstrates how to selectively download specific files from a Hugging Face model repository:

1. **File Definition**: First, a list of commonly required files for transformer models is defined, with comments explaining each file's purpose:
   - Vocabulary files for tokenization
   - Configuration files for model architecture
   - Tokenizer files for text preprocessing
   - Model weights in different formats (PyTorch and SafeTensors)

2. **Selective Download**: The code iterates through each file in the list and:
   - Attempts to download it using `hf_hub_download()`
   - Specifies the model repository via `repo_id=hf_reference`
   - Saves files to a local directory structure based on the model name
   - Prints the local path where each file is saved

3. **Error Handling**: The try-except block catches and reports any download failures, allowing the process to continue even if certain files aren't available for the specific model.


In [ ]:
# List of required files
required_files = [
    "vocab.txt",          # Vocabulary file (if applicable)
    "vocab.json",          # Vocabulary file (if applicable)
    "config.json",        # Model configuration
    "tokenizer.json",     # Tokenizer configuration (if applicable)
    "merges.txt",         # BPE merge rules file (if applicable)
    "pytorch_model.bin",  # Model weights
    "model.safetensors",  # Alternative model weights format
]


# Download only the required files
for file_name in required_files:
    try:
        print()
        print(f"Attempting to download: {file_name}")
        local_path = hf_hub_download(repo_id=hf_reference, filename=file_name, local_dir=f"models/{hf_reference.split('/')[1]}")
        print(f"Saved to: {local_path}")
    except Exception as e:
        print(f"Could not download {file_name}: {e}")





Attempting to download: vocab.txt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt: 0.00B [00:00, ?B/s]

Saved to: models/bert-base-multilingual-uncased-sentiment/vocab.txt

Attempting to download: vocab.json
Could not download vocab.json: 404 Client Error. (Request ID: Root=1-68e0def9-4b30531e604403c707f5977a;c90a1d33-96de-4cc0-aa9b-176fcf3a5f1f)

Entry Not Found for url: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/resolve/main/vocab.json.

Attempting to download: config.json


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

Saved to: models/bert-base-multilingual-uncased-sentiment/config.json

Attempting to download: tokenizer.json
Could not download tokenizer.json: 404 Client Error. (Request ID: Root=1-68e0def9-060ba24425a4398e23a9bde1;40ecf11f-972d-44b0-80f2-c579a4da4acb)

Entry Not Found for url: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/resolve/main/tokenizer.json.

Attempting to download: merges.txt
Could not download merges.txt: 404 Client Error. (Request ID: Root=1-68e0def9-2f6513a66b84615c34295fc0;f45537b1-cd6b-4a4e-89fb-b95873092212)

Entry Not Found for url: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/resolve/main/merges.txt.

Attempting to download: pytorch_model.bin


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

Saved to: models/bert-base-multilingual-uncased-sentiment/pytorch_model.bin

Attempting to download: model.safetensors


model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Saved to: models/bert-base-multilingual-uncased-sentiment/model.safetensors



# Creating Sentiment Analysis Pipelines


This code initializes two sentiment analysis pipelines using Hugging Face's `transformers` library:

1. **Cached Model Pipeline**:
   - `hf_model_cache` uses the model identifier directly (`hf_reference`)
   - When this pipeline runs, it will first check the default Hugging Face cache directory on your system
   - If not found in cache, it automatically downloads the model from Hugging Face Hub

2. **Local Model Pipeline**:
   - `hf_model_local` uses the previously downloaded model files
   - Points to the local directory where model files were saved earlier
   - Loads the model from the local files rather than downloading or using cache
   - Path is constructed by extracting just the model name from the reference

Both pipelines provide the same sentiment analysis functionality but differ in where they source the model files from, allowing flexibility between network-dependent and offline usage.


In [ ]:
hf_model_cache = pipeline("sentiment-analysis", model=hf_reference)
hf_model_local = pipeline("sentiment-analysis", model=f"models/{hf_reference.split('/')[1]}")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu



# Testing the Sentiment Analysis Model


This code demonstrates how to use the local sentiment analysis pipeline:

1. **Input Text**: Defines a medical text about high blood sugar and blood pressure readings

2. **Model Inference**: Passes the text to the previously initialized local sentiment analysis model (`hf_model_local`)

3. **Output Display**: Prints the classification results, which typically include:
   - The predicted sentiment label (POSITIVE or NEGATIVE)
   - A confidence score indicating the model's certainty
   
Note that this medical statement is factual rather than emotionally charged, so the model's sentiment prediction may not be particularly meaningful in this context. Sentiment analysis models work best with text that expresses opinions or emotional reactions.


In [ ]:
text = "The shoes fit very well, I just dont like fabric"
print(hf_model_local(text))

[{'label': '4 stars', 'score': 0.4492033123970032}]


In [ ]:
text = "The students from this school score very high marks"
print(hf_model_local(text))

[{'label': '1 star', 'score': 0.2396576702594757}]


checking the labels

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


In [ ]:
model.config.id2label # dont learn; refer docs

{0: '1 star', 1: '2 stars', 2: '3 stars', 3: '4 stars', 4: '5 stars'}


# Serialize and Save Model Information from Hugging Face Hub


This code demonstrates how to retrieve, serialize, and save detailed model information from the Hugging Face Hub:

1. **Serialization Function**: The `serialize_object()` function handles complex objects recursively:
   - Converts datetime objects to ISO format strings
   - Transforms objects with `__dict__` attributes into dictionaries
   - Processes nested lists and dictionaries
   - Preserves primitive data types

2. **API Interaction**: Creates an instance of the Hugging Face API client

3. **Model Information**: Fetches comprehensive metadata about the specified model using `api.model_info()`

4. **File Operations**:
   - Extracts the model name from the reference path
   - Creates a JSON file named after the model
   - Serializes the model information and writes it to the file

This allows for local storage of model metadata for later reference or analysis, particularly useful for model governance, versioning, and documentation purposes.


In [ ]:
def serialize_object(obj):
    """
    Helper function to serialize custom objects like EvalResult.
    Converts objects with __dict__ attribute to dictionaries and handles datetime objects.
    """
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to ISO 8601 string
    elif hasattr(obj, "__dict__"):
        return {key: serialize_object(value) for key, value in obj.__dict__.items()}
    elif isinstance(obj, list):
        return [serialize_object(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_object(value) for key, value in obj.items()}
    else:
        return obj  # Return the value as-is for primitive types

api = HfApi()
with open(f"models/{hf_reference.split('/')[1]}.json", "w") as json_file:
    json_file.write(json.dumps(serialize_object(api.model_info(hf_reference))))
